In [3]:
line = "the entire movie is filled with deja vu moments ."
tokens = line.split()
print(tokens)
print(type(tokens))

['the', 'entire', 'movie', 'is', 'filled', 'with', 'deja', 'vu', 'moments', '.']
<class 'list'>


In [4]:
sentence = " ".join(tokens)
print(sentence)
print(type(sentence))

the entire movie is filled with deja vu moments .
<class 'str'>


In [1]:
import torch

In [2]:
y_data = [3, 1, 2, 3, 2]
labals = torch.LongTensor(y_data)
print(labals)

tensor([3, 1, 2, 3, 2])


In [3]:
from sklearn.preprocessing import LabelEncoder

In [5]:
encoder = LabelEncoder()
encoder.fit(y_data)
y = encoder.transform(y_data)
labals = torch.LongTensor(y)
print(labals.size())
print(labals)

torch.Size([5])
tensor([2, 0, 1, 2, 1])


In [6]:
import torch
from torch.nn import functional as f
 
x = torch.arange(0, 1*300*10*1).float()
x = x.view(1,300,10,1)
print(x.shape)
x1 = f.unfold(x, kernel_size=(3,1), padding=(3 // 2, 0))
print(x1.shape)
B, C_kh_kw, L = x1.size()
x1 = x1.permute(0, 2, 1)
x1 = x1.view(B, L, -1, 3, 3)
print(x1.shape)

torch.Size([1, 300, 10, 1])
torch.Size([1, 900, 10])
torch.Size([1, 10, 100, 3, 3])


In [11]:
import torch.nn.functional as F
q = torch.arange(0,1*6*50*1*10)
q = q.view(1,6,50,1,10)
k = torch.arange(0,1*6*50*3*10)
k = k.view(1,6,50,3,10)
out = torch.matmul(q, k.transpose(-1,-2))
out2 = q * k
print(out.shape)
print(out2.shape)

torch.Size([1, 6, 50, 1, 3])
torch.Size([1, 6, 50, 3, 10])


In [26]:
a = torch.Tensor(1,2)
b = torch.Tensor(3,2,1)
c = torch.matmul(a,b)
print(c.shape)

torch.Size([3, 1, 1])


In [36]:
a = torch.Tensor([[[1,2],[3,4]],[[1,2],[3,4]],[[1,2],[3,4]]])
b = torch.Tensor([[[1,2],[3,4]],[[1,2],[4,2]],[[1,2],[1,1]]])
c = torch.matmul(a,b)
print(c.shape)
print(c)
print(c.sum(0))

torch.Size([3, 2, 2])
tensor([[[ 7., 10.],
         [15., 22.]],

        [[ 9.,  6.],
         [19., 14.]],

        [[ 3.,  4.],
         [ 7., 10.]]])
tensor([[19., 20.],
        [41., 46.]])


In [24]:
m = torch.nn.Softmax(dim=-1)
input = torch.Tensor([[1,2,3],[4,5,6]])
output = m(input)
print(output)

tensor([[0.0900, 0.2447, 0.6652],
        [0.0900, 0.2447, 0.6652]])


In [41]:
x = torch.zeros(10, 1).byte()
print(x)

tensor([[0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0]], dtype=torch.uint8)


In [44]:
a = torch.arange(10, dtype=torch.long).view(1,10).unsqueeze(2)
print(a.shape)

torch.Size([1, 10, 1])


In [2]:
import torch

In [4]:
mask = torch.Tensor([[1,1,1,0,0]])
print(mask.shape)
mask = (mask.eq(False))
print(mask)
smask = torch.cat([torch.zeros(1, 1).byte().to(mask), mask], 1)
print(smask)

test = torch.zeros(1, 1).byte().to(mask)
print(test)

ex_mask = mask[:, None, :, None]
print(ex_mask.shape)

torch.Size([1, 5])
tensor([[False, False, False,  True,  True]])
tensor([[False, False, False, False,  True,  True]])
tensor([[False]])
torch.Size([1, 1, 5, 1])


In [4]:
def seq_len_to_mask(seq_len, max_len=None):
    r"""
    将一个表示sequence length的一维数组转换为二维的mask，不包含的位置为0。
    转变 1-d seq_len到2-d mask.
    .. code-block::
    
        >>> seq_len = torch.arange(2, 16)
        >>> mask = seq_len_to_mask(seq_len)
        >>> print(mask.size())
        torch.Size([14, 15])
        >>> seq_len = np.arange(2, 16)
        >>> mask = seq_len_to_mask(seq_len)
        >>> print(mask.shape)
        (14, 15)
        >>> seq_len = torch.arange(2, 16)
        >>> mask = seq_len_to_mask(seq_len, max_len=100)
        >>>print(mask.size())
        torch.Size([14, 100])
    :param np.ndarray,torch.LongTensor seq_len: shape将是(B,)
    :param int max_len: 将长度pad到这个长度。默认(None)使用的是seq_len中最长的长度。但在nn.DataParallel的场景下可能不同卡的seq_len会有
        区别，所以需要传入一个max_len使得mask的长度是pad到该长度。
    :return: np.ndarray, torch.Tensor 。shape将是(B, max_length)， 元素类似为bool或torch.uint8
    """
    if isinstance(seq_len, np.ndarray):
        assert len(np.shape(seq_len)) == 1, f"seq_len can only have one dimension, got {len(np.shape(seq_len))}."
        max_len = int(max_len) if max_len else int(seq_len.max())
        broad_cast_seq_len = np.tile(np.arange(max_len), (len(seq_len), 1))
        mask = broad_cast_seq_len < seq_len.reshape(-1, 1)

    elif isinstance(seq_len, torch.Tensor):
        assert seq_len.dim() == 1, f"seq_len can only have one dimension, got {seq_len.dim() == 1}."
        batch_size = seq_len.size(0)
        max_len = int(max_len) if max_len else seq_len.max().long()
        broad_cast_seq_len = torch.arange(max_len).expand(batch_size, -1).to(seq_len)
        mask = broad_cast_seq_len.lt(seq_len.unsqueeze(1))
    else:
        raise TypeError("Only support 1-d numpy.ndarray or 1-d torch.Tensor.")

    return mask

In [6]:
import torch
seq_len = torch.full((32,), 10)
print(seq_len)

tensor([10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
        10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
        10., 10., 10., 10.])


In [8]:
import numpy as np
def seq_len_to_mask(seq_len, max_len=None):
    r"""
    将一个表示sequence length的一维数组转换为二维的mask，不包含的位置为0。
    转变 1-d seq_len到2-d mask.
    .. code-block::
    
        >>> seq_len = torch.arange(2, 16)
        >>> mask = seq_len_to_mask(seq_len)
        >>> print(mask.size())
        torch.Size([14, 15])
        >>> seq_len = np.arange(2, 16)
        >>> mask = seq_len_to_mask(seq_len)
        >>> print(mask.shape)
        (14, 15)
        >>> seq_len = torch.arange(2, 16)
        >>> mask = seq_len_to_mask(seq_len, max_len=100)
        >>>print(mask.size())
        torch.Size([14, 100])
    :param np.ndarray,torch.LongTensor seq_len: shape将是(B,)
    :param int max_len: 将长度pad到这个长度。默认(None)使用的是seq_len中最长的长度。但在nn.DataParallel的场景下可能不同卡的seq_len会有
        区别，所以需要传入一个max_len使得mask的长度是pad到该长度。
    :return: np.ndarray, torch.Tensor 。shape将是(B, max_length)， 元素类似为bool或torch.uint8
    """
    if isinstance(seq_len, np.ndarray):
        assert len(np.shape(seq_len)) == 1, f"seq_len can only have one dimension, got {len(np.shape(seq_len))}."
        max_len = int(max_len) if max_len else int(seq_len.max())
        broad_cast_seq_len = np.tile(np.arange(max_len), (len(seq_len), 1))
        mask = broad_cast_seq_len < seq_len.reshape(-1, 1)

    elif isinstance(seq_len, torch.Tensor):
        assert seq_len.dim() == 1, f"seq_len can only have one dimension, got {seq_len.dim() == 1}."
        batch_size = seq_len.size(0)
        max_len = int(max_len) if max_len else seq_len.max().long()
        broad_cast_seq_len = torch.arange(max_len).expand(batch_size, -1).to(seq_len)
        mask = broad_cast_seq_len.lt(seq_len.unsqueeze(1))
    else:
        raise TypeError("Only support 1-d numpy.ndarray or 1-d torch.Tensor.")

    return mask

In [13]:
import torch
seq_len = torch.full((32,), 10)
print(seq_len)
mask = seq_len_to_mask(seq_len)
print(mask)

tensor([10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
        10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
        10., 10., 10., 10.])
torch
tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, 

In [11]:
a = torch.arange(10)
print(a)
a = a.expand(32,-1)
print(a)
a = a.lt()

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
tensor([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 

In [24]:
def getLen_in_batch(sequences, device):
    result = []
    for sequence in sequences:
        cnt = 0
        for num in sequence:
            if num != 0:
                cnt += 1
        result.append(cnt)
    return torch.Tensor(result)

In [26]:
device = torch.device('cpu')
x = torch.Tensor([[1,2,0,0,0],[1,2,4,5,8]])
y = getLen_in_batch(x, device)
print(y)

tensor([2., 5.])


In [23]:
a = [1, 2, 3, 4]
b = torch.Tensor(a)
print(a)

[1, 2, 3, 4]
